In [11]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [12]:
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df = df[df['y'].notnull()]
df.shape

(498, 16)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [14]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Limpeza da data

In [15]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [16]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [17]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [18]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [19]:
features.head()

,views,views_por_dia
0,28028,61.464912
1,1131,2.960733
2,1816,8.446512
3,1171,10.455357
4,1228,3.336957


In [20]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = df_limpo['date'] >= "2019-04-01"

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (270, 2), (228,), (270,))

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [27]:
title_bow_train.shape

(228, 193)

In [30]:
1 - 1277/(228*193)

0.9709799109171894

In [28]:
title_bow_train

<228x193 sparse matrix of type '<class 'numpy.float64'>'
	with 1277 stored elements in Compressed Sparse Row format>

In [ ]:
hstack - [1 2]     [3 4]   -> [1 2 3 4] - 1x4

vstack - [1 2]     [3 4]   -> [1 2]
                              [3 4] - 2x2

In [32]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [33]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((228, 195), (270, 195))

In [34]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=6, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [35]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [36]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [37]:
average_precision_score(yval, p)

0.1918043901336543

In [38]:
roc_auc_score(yval, p)

0.5848024316109421

In [ ]:
ap 0.17683995211083103, auc 0.6036474164133738 - mindf=1
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2

# 5 Active learning

70 exemplos que o modelo tenha dificuldade
30 exemplos aleatoriamente

In [39]:
df_unlabeled = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df_unlabeled = df_unlabeled[df_unlabeled['y'].isnull()].dropna(how='all')
df_unlabeled.shape

(674, 16)

In [40]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w


In [41]:
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['watch-title']

In [42]:
clean_date = df_unlabeled['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo_u['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [43]:
df_limpo_u.head()

,title,date
501,Kaggle Mercari Price Suggestion Challenge (1 p...,2018-11-02
502,OpenAI Gym and Python for Q-learning - Reinfor...,2018-10-14
503,"Dashboarding with Notebooks, Day 1: What infor...",2018-12-17
504,How To Get US- American Company H1 Visa To Get...,2019-11-23
505,Platform Overview - Machine Learning,2019-05-21


In [44]:
views = df_unlabeled['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo_u['views'] = views

In [45]:
features_u = pd.DataFrame(index=df_limpo_u.index)

In [46]:
features_u['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo_u['date']) / np.timedelta64(1, 'D')
features_u['views'] = df_limpo_u['views']
features_u['views_por_dia'] = features_u['views'] / features_u['tempo_desde_pub']
features_u = features_u.drop(['tempo_desde_pub'], axis=1)

In [47]:
features_u.head()

,views,views_por_dia
501,2167,5.472222
502,20378,49.103614
503,10435,29.729345
504,7,0.700000
505,4298,21.928571


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_u = df_limpo_u['title']
title_bow_u = title_vec.transform(title_u)


In [49]:
title_bow_u

<674x193 sparse matrix of type '<class 'numpy.float64'>'
	with 3079 stored elements in Compressed Sparse Row format>

In [50]:
Xu_wtitle = hstack([features_u, title_bow_u])

In [51]:
Xu_wtitle

<674x195 sparse matrix of type '<class 'numpy.float64'>'
	with 4409 stored elements in COOrdinate format>

In [52]:
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [53]:
df_unlabeled['p'] = pu

In [54]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w,0.093


In [58]:
mask_u = (df_unlabeled['p'] >= 0.26) & (df_unlabeled['p'] <= 1.)
mask_u.sum()

69

In [59]:
mask_u

501     False
502     False
503     False
504     False
505      True
        ...  
1179    False
1180    False
1181    False
1182    False
1183    False
Name: p, Length: 674, dtype: bool

In [56]:
df_unlabeled[mask_u]#.sort_values("p")

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502
507,Kaggle Meetup: Ship Detection Challenge,NaN,504 visualizações,Publicado em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Meetup: Ship Detection Challenge,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,504 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/QXEy4rdLsDw/maxresdefau...,1280.0,720.0,Video from the 2018-11-29 meetup. Kaggle page:...,1280.0,720.0,learn data science,/channel/UCJhW_16uxALr0X4olEW2p5A,0.455
535,Identifying art through machine learning at MoMA,NaN,9.476 visualizações,Publicado em 7 de mar. de 2018,Entretenimento,#MachineLearning #Musuem #Art\n\n\n\n Ident...,Google Arts & Culture\n\n\n\n\n\n\n\n\n\n\n\n\...,9.476 visualizações\n\n\n\n\n\n\n\n49\n\nGosto...,https://i.ytimg.com/vi/SLBqVOnn9Mo/maxresdefau...,1280.0,720.0,"The Art Recognizer, built in collaboration wit...",1280.0,720.0,museum of modern art,/channel/UCGn7dlcAmH44GqycKa_3ssA,0.510
557,Deep Learning - Computerphile,NaN,131.286 visualizações,Publicado em 6 de jun. de 2018,Educação,Deep Learning - Computerphile,Computerphile\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCar...,131.286 visualizações\n\n\n\n\n\n\n\n3.700\n\n...,https://i.ytimg.com/vi/TJlAxW-2nmI/maxresdefau...,1280.0,720.0,Deep Learning with Convolutional Neural Networ...,1280.0,720.0,Neural Network,/channel/UC9-y-6csu5WGm29I7JiwpnA,0.453
705,Kaggle Meetup: CANSSI NCSC Ferry Delays,NaN,124 visualizações,Publicado em 3 de out. de 2019,Ciência e tecnologia,Kaggle Meetup: CANSSI NCSC Ferry Delays,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,124 visualizações\n\n\n\n\n\n\n\n2\n\nGostou d...,https://i.ytimg.com/vi/aOGnEHywIBc/maxresdefau...,1280.0,720.0,Presenter: Matt Tourond Kaggle page: https://w...,1280.0,720.0,machine learning,/channel/UCJhW_16uxALr0X4olEW2p5A,0.464
718,Kaggle Meetup: Human Protein Atlas,NaN,226 visualizações,Publicado em 8 de fev. de 2019,Ciência e tecnologia,Kaggle Meetup: Human Protein Atlas,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,226 visualizações\n\n\n\n\n\n\n\n5\n\nGostou d...,https://i.ytimg.com/vi/bAgjqdQ0VE8/maxresdefau...,1280.0,720.0,Kaggle page: https://www.kaggle.com/c/human-pr...,1280.0,720.0,kaggle,/channel/UCJhW_16uxALr0X4olEW2p5A,0.487
744,Data Science Specialization at IFMR-GSB,NaN,207 visualizações,Publicado em 27 de dez. de 2018,Educação,Data Science Specialization at IFMR-GSB,IFMR GSB KREA University\n\n\n\n\n\n\n\n\n\n\n...,207 visualizações\n\n\n\n\n\n\n\n3\n\nGostou d...,https://i.ytimg.com/vi/crUfuo9GXIc/maxresdefau...,1280.0,720.0,,1280.0,720.0,NaN,/channel/UCAjwb2trX_m_6hobx02MV9g,0.469
904,Kaggle Meetup: AdTracking Fraud Detection,NaN,372 visualizações,Publicado em 1 de jun. de 2018,Ciência e tecnologia,Kaggle Meetup: AdTracking Fraud Detection,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,372 visualizações\n\n\n\n\n\n\n\n11\n\nGostou ...,https://i.ytimg.com/vi/kzkQTPeZPQ4/maxresdefau...,1280.0,720.0,https://www.meetup.com/LearnDataScience/events...,1280.0,720.0,NaN,/channel/UCJhW_16uxALr0X4olEW2p5A,0.521
923,Shri Debasish Sen at 7th International Data Sc...,NaN,5 visualizações,Publicado em 23 de nov. de 2019,Ciência e tecnologia,Shri Debasish Sen at 7th International Data Sc...,Data Science Foundation\n\n\n\n\n\n\n\n\n\n\n\...,5 visualizações\n\n\n\n\nGostou deste vídeo?\n...,https://i.ytimg.com/vi/m0NTnHGSfkA/maxresdefau

In [60]:
dificeis = df_unlabeled[mask_u]

In [63]:
aleatorios = df_unlabeled[~mask_u].sample(31, random_state=0)

In [104]:
pd.concat([dificeis, aleatorios]).to_csv("active_label1.csv")

In [61]:
dificeis.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502
507,Kaggle Meetup: Ship Detection Challenge,NaN,504 visualizações,Publicado em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Meetup: Ship Detection Challenge,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,504 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/QXEy4rdLsDw/maxresdefau...,1280.0,720.0,Video from the 2018-11-29 meetup. Kaggle page:...,1280.0,720.0,learn data science,/channel/UCJhW_16uxALr0X4olEW2p5A,0.455
519,flowlightさん『TalkingData AdTracking Fraud Detec...,NaN,1.707 visualizações,Publicado em 12 de mai. de 2018,Pessoas e blogs,flowlightさん『TalkingData AdTracking Fraud Detec...,Takami Sato\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarrega...,1.707 visualizações\n\n\n\n\n\n\n\n17\n\nGosto...,https://i.ytimg.com/vi/RKXUEJVJJ-o/maxresdefau...,1280.0,720.0,資料 https://www.slideshare.net/TakanoriHayashi3...,1280.0,720.0,kaggle,/channel/UCiECS_auJLNpFsvjTi1WuxQ,0.283
527,Reinforcement Learning with TensorFlow and Uni...,NaN,545 visualizações,Publicado em 22 de nov. de 2019,Ciência e tecnologia,Reinforcement Learning with TensorFlow and Uni...,Google Developers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,545 visualizações\n\n\n\n\n\n\n\n13\n\nGostou ...,https://i.ytimg.com/vi/S-MbpQiwfls/maxresdefau...,1280.0,720.0,"Dan Goncharov, Head of 42 Robotics GDG Fremont...",1280.0,720.0,Dan Goncharov,/channel/UC_x5XG1OV2P6uZZ5FSM9Ttw,0.261
535,Identifying art through machine learning at MoMA,NaN,9.476 visualizações,Publicado em 7 de mar. de 2018,Entretenimento,#MachineLearning #Musuem #Art\n\n\n\n Ident...,Google Arts & Culture\n\n\n\n\n\n\n\n\n\n\n\n\...,9.476 visualizações\n\n\n\n\n\n\n\n49\n\nGosto...,https://i.ytimg.com/vi/SLBqVOnn9Mo/maxresdefau...,1280.0,720.0,"The Art Recognizer, built in collaboration wit...",1280.0,720.0,museum of modern art,/channel/UCGn7dlcAmH44GqycKa_3ssA,0.510


In [64]:
aleatorios.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
929,Mathematics for Machine Learning Full Course |...,NaN,95.544 visualizações,Publicado em 14 de ago. de 2018,Educação,Mathematics for Machine Learning Full Course |...,Geek's Lesson\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,95.544 visualizações\n\n\n\n\n\n\n\n2.806\n\nG...,https://i.ytimg.com/vi/m998PdOCFcY/maxresdefau...,1280.0,720.0,This course offers a brief introduction to the...,640.0,360.0,calcu,/channel/UCKXx22vOENUyHrVAADq7Z_g,0.058
608,初めてkaggleのkernelを使う方へ、使い方を説明します,NaN,1.718 visualizações,Publicado em 31 de jan. de 2019,Ciência e tecnologia,#curry_ch\n\n\n\n 初めてkaggleのkernelを使う方へ、使い方...,カレーちゃん\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.718 visualizações\n\n\n\n\n\n\n\n43\n\nGosto...,https://i.ytimg.com/vi/W8KybMGVFzk/hqdefault.jpg,480.0,360.0,KaggleのKernelではブラウザ上から、PythonやＲのコードを書いて動かすことがで...,960.0,720.0,curry_ch,/channel/UCF_3GKr98gkLer3iXJHA0RQ,0.067
990,How Machine Learning Works (Simple & Short- 20...,NaN,30 visualizações,Publicado em 22 de nov. de 2019,Ciência e tecnologia,How Machine Learning Works (Simple & Short- 20...,Factual Funda\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,30 visualizações\n\n\n\n\n\n\n\n16\n\nGostou d...,https://i.ytimg.com/vi/pW3ty4zx1iQ/maxresdefau...,1280.0,720.0,In this video we are going to discuss about Ma...,1280.0,720.0,factualfunda,/channel/UCXZvnryrHB4yiyVb0SUpWqQ,0.197
1083,Kaggle Live-Coding: Deep Learning with GPUs (p...,NaN,2.576 visualizações,Publicado em 18 de jun. de 2018,Ciência e tecnologia,Kaggle Live-Coding: Deep Learning with GPUs (p...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,2.576 visualizações\n\n\n\n\n\n\n\n58\n\nGosto...,https://i.ytimg.com/vi/u9pIhOay8Fw/maxresdefau...,1280.0,720.0,Join Kaggle Data Scientist Rachael as she star...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.132
939,1/5 攻占 Kaggle $25000 奖池：和 1873 支数据大神队伍一起奋战写出适合...,NaN,438 visualizações,Publicado em 7 de jan. de 2019,Ciência e tecnologia,1/5 攻占 Kaggle $25000 奖池：和 1873 支数据大神队伍一起奋战写出适合...,Data Application Lab\n\n\n\n\n\n\n\n\n\n\n\n\n...,438 visualizações\n\n\n\n\n\n\n\n11\n\nGostou ...,https://i.ytimg.com/vi/mfsFo9vzopE/maxresdefau...,1280.0,720.0,Kaggle 是世界上最 NB 的数据科学家以及机器学习侠们的聚集地。最开始 Kaggle ...,1280.0,720.0,data challenge,/channel/UCa8NLpvi70mHVsW4J_x9OeQ,0.119
